### Chain example with custom tools

In [53]:
#setup
import openai
import os
from pprint import pprint
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.chat_models import AzureChatOpenAI
from typing import Optional, Type
from langchain.tools import BaseTool
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
# openai.api_version = "2023-03-15-preview"
openai.api_version = "2023-05-15"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [54]:
# create llm instance
llm = AzureChatOpenAI(deployment_name=deployment_name_gpt)

In [55]:
#define embedding and creating the instnace of azure cognitive search
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=deployment_name_ada, chunk_size=1)
index_name: str = "facts-about-snakes"
azure_search_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    semantic_configuration_name="default",
    embedding_function=embeddings.embed_query,
)

In [56]:
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to get exisitng data on snakes from azure cognitive seach"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return azure_search_store.similarity_search(
            query=query, 
            k=4,
            search_type="semantic_hybrid"
        )

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [57]:
acs_tools = [CustomSearchTool()]
agent = initialize_agent(
    acs_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [58]:
# chain 1
chain_one = agent

In [95]:
template = """You are a text summrizer. Your job is to summarize the inputed data and provide the content in html format\n
Provide a good tite for the html document and the summary content in the div\n 

Input:
{input}

"""
prompt_template = PromptTemplate(input_variables=["input"], template=template)

In [60]:
chain_two = LLMChain(llm=llm, prompt=prompt_template) 

In [61]:
overall_chain = SimpleSequentialChain(
                  chains=[chain_one, chain_two],
                  verbose=True)

In [96]:
# for viewing html output
from IPython.core.display import display, HTML


/tmp/ipykernel_64210/3372027314.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [97]:
question = "What are the venomous snakes in North Carolina?"

result = overall_chain.run(question)
display(HTML(result))



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
I need to find existing data on venomous snakes in North Carolina.
Action: custom_search
Action Input: "venomous snakes North Carolina"
Observation: [Document(page_content='Few creatures are as widely misunderstood as snakes.  \nFueled by myths and old-wives tales, many people \nfear snakes and worry for the safety of people and pets when snakes are present. In reality, snakes are \nshy creatures that pose little to no threat to us when  \nleft alone. Snakes are an important part of our environment, keeping populations of pests such as rodents, slugs, and insects in check. Plus, snakes are a food resource to other animals such as foxes, raccoons, bears, eagles, hawks, and owls.    \ncoexist  with\nSnakes\nSnake Quick Facts: \nHow many snakes in North Carolina are \nVenomous?\nOut of the thirty-eight different species of snakes in North \nCarolina, only six are venomous. Venomous snakes include \nthe

In [98]:
question = "Why do snakes shed their skins?"

result = overall_chain.run(question)
display(HTML(result))



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
Snakes shedding their skins is a common behavior among reptiles, so I should start by researching this behavior.
Action: custom_search
Action Input: "reasons why snakes shed their skins"
Observation: [Document(page_content='Eastern Coral Snake. The Copperhead is the most common \nand widespread venomous snake in North Carolina, occur-\nring in both rural and urban environments. Four of the six \nare protected species in North Carolina, and as such, should \nnot be handled or disturbed: the Timber and Pigmy Rattle-\nsnakes are both Species of Special Concern, while the \nEastern Diamondback Rattlesnake and Eastern Coral Snake \nare both Endangered.  \nWhy do snakes shed their skins?\nAs a snake grows, its body increas es in size, b ut its skin \ndoes not grow with it. Therefore, as they grow, snakes \nmust replace their sma ller skins with lar ger ones. The act \nof shedding old s kin also removes  a

In [99]:
question = "List the tips for Preventing Conflicts with Snakes"

result = overall_chain.run(question)
display(HTML(result))



> Entering new SimpleSequentialChain chain...


> Entering new AgentExecutor chain...
I should search for a reputable source on snake safety tips
Action: custom_search
Action Input: "snake safety tips" in Azure Cognitive Search
Observation: [Document(page_content='snakes (and other wildlife) out of your living space. \n•   BE AWARE OF YOUR SURROUNDINGS. As you \ngo for walks outside, do yard work, or work in the \ngarden, be aware that snakes could be in the area. \nBeing prepared for the possibility of encountering a snake and watching where you step and place your hands can minimize the chance of you disturbing or stepping on a snake.', metadata={'source': '../data/snakes.pdf', 'page': 1, 'captions': {'text': 'snakes (and other wildlife) out of your living space. \n•   BE AWARE OF YOUR SURROUNDINGS. As you \ngo for walks outside, do yard work, or work in the \ngarden, be aware that snakes could be in the area. \nBeing prepared for the possibility of encountering a snake and watchin